In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
from sklearn.preprocessing import LabelEncoder
pd.set_option('mode.chained_assignment', None)
from tqdm import tqdm

In [3]:
parent_dir = os.path.abspath(os.path.join(os.getcwd(), '../'))
subdir = 'utils'
path = os.path.join(parent_dir, subdir)
sys.path.append(path)
from tqdm import tqdm

In [4]:
path_to_directory = os.path.abspath('../data')
train_data = 'train.csv'
test_data = 'test.csv'
train_df = pd.read_csv(os.path.join(path_to_directory, train_data) )
test_df = pd.read_csv(os.path.join(path_to_directory, test_data) )

In [5]:
train_df = train_df.fillna(0)

# Decision Treest

Recursive partitioning. Regularisation reduce the depth of the tree.

Advantages:
- white box models

Disadvantages:

In [6]:
# Decision trees

class Decision_tree():
    def __init__(self, data, target) -> None:
        self.data = data
        self.target = target
        self.set = set()
        
    def all_splits(self):
        columns = [col for col in self.data.columns if col not in ['PassengerId', self.target, 'Name', 'Ticket'] ]
        gini_values = {}
        for col  in columns:
            for val in self.data[col].unique():
                gini_weighted = self.split(col, val)
                gini_values[col + '_split_' + str(val)] = gini_weighted
                
        # Find the minimum value of the dictionary and update the self.data
        min_key = min(gini_values, key=gini_values.get)
        col, val = min_key.split('_split_')
        print(col, val)
        return gini_values
        
            
    def split(self, col, val):
        
        # Calculate probability for each node
        cls1 = self.data[self.data[col] == val]
        cls2 = self.data[self.data[col] != val]
        prob_list, n1, n2 = self.calculate_probability(cls1, cls2)
        
        # Calculate gini_index for each node
        g1, g2 = self.gini_index(prob_list)
        
        # Return weighted gini-impuritiny index
        return self.gini_impurity_for_split(g1, g2, n1, n2)
    
    def calculate_probability(self, cls1, cls2):
        ''''''
        prob_cls1 = []
        prob_cls2 = []
        n1 = len(cls1)
        n2 = len(cls2)
        for val in cls1[self.target].unique():
            temp_data = len(cls1[cls1[self.target] == val].copy(deep=True))
            prob_cls1.append(temp_data / n1)

        for val in cls2[self.target].unique():
            value2 = len(cls2[cls2[self.target] == val].copy(deep=True))
            prob_cls2.append(value2 / n2)
        return [prob_cls1, prob_cls2], n1, n2
        
    def gini_index(self, prob_list):
        g1 =  1 - np.sum([element **2 for element in prob_list[0]])
        g2 =  1 - np.sum([element **2 for element in prob_list[1]])
        return g1, g2
        
    def gini_impurity_for_split(self, g1, g2, n1, n2):
        return n1/(n1+n2) * g1 + n2/(n1+n2) * g2
        

In [7]:
decision_tree = Decision_tree(train_df, 'Survived')
gini_dict = decision_tree.all_splits()

Pclass 3
Pclass 1
Pclass 2
Sex male
Sex female
Age 22.0
Age 38.0
Age 26.0
Age 35.0
Age 0.0
Age 54.0
Age 2.0
Age 27.0
Age 14.0
Age 4.0
Age 58.0
Age 20.0
Age 39.0
Age 55.0
Age 31.0
Age 34.0
Age 15.0
Age 28.0
Age 8.0
Age 19.0
Age 40.0
Age 66.0
Age 42.0
Age 21.0
Age 18.0
Age 3.0
Age 7.0
Age 49.0
Age 29.0
Age 65.0
Age 28.5
Age 5.0
Age 11.0
Age 45.0
Age 17.0
Age 32.0
Age 16.0
Age 25.0
Age 0.83
Age 30.0
Age 33.0
Age 23.0
Age 24.0
Age 46.0
Age 59.0
Age 71.0
Age 37.0
Age 47.0
Age 14.5
Age 70.5
Age 32.5
Age 12.0
Age 9.0
Age 36.5
Age 51.0
Age 55.5
Age 40.5
Age 44.0
Age 1.0
Age 61.0
Age 56.0
Age 50.0
Age 36.0
Age 45.5
Age 20.5
Age 62.0
Age 41.0
Age 52.0
Age 63.0
Age 23.5
Age 0.92
Age 43.0
Age 60.0
Age 10.0
Age 64.0
Age 13.0
Age 48.0
Age 0.75
Age 53.0
Age 57.0
Age 80.0
Age 70.0
Age 24.5
Age 6.0
Age 0.67
Age 30.5
Age 0.42
Age 34.5
Age 74.0
SibSp 1
SibSp 0
SibSp 3
SibSp 4
SibSp 2
SibSp 5
SibSp 8
Parch 0
Parch 1
Parch 2
Parch 5
Parch 3
Parch 4
Parch 6
Fare 7.25
Fare 71.2833
Fare 7.925
Fare 53.1
Fare 8